In [1]:
import pandas as pd
import numpy as np

from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
from sklearn.metrics import pairwise_distances

import nltk
import string

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from src import nlp_topic_utils

# Wall Street Forum Topic Modeling

## Data Import and Tf-Idf Vectorization

In [2]:
wallstreet = pd.read_csv('data/wallstreet_master.csv')

In [3]:
docs_raw = wallstreet['processed_text']
n_topics = 20
max_doc_freq = 0.7
min_doc_freq = 4
max_features = 20000
ngram_rng = [1, 2]
max_iterations = 25

In [4]:
docs_vectorized, vectorizer = nlp_topic_utils.tfidf_vectorizer(docs_raw, min_doc_freq, max_doc_freq, max_features, ngram_rng)

## PCA & Clustering

In [6]:
prcnt_var_to_keep = 0.90
random_state = 3

pca = PCA(n_components=prcnt_var_to_keep, random_state=random_state)
docs_reduced = pca.fit_transform(docs_vectorized.toarray())
docs_reduced.shape

MemoryError: Unable to allocate 6.76 GiB for an array with shape (45372, 20000) and data type float64

---
## K-Means

In [ ]:
def calc_sum_sqrd_errors(docs_vectorized, kmax, increment):
    
    sum_sqrd_errs = []
    for k in range(1, kmax + increment, increment):
        kmeans = KMeans(n_clusters=k).fit(docs_vectorized)
        centroids = kmeans.cluster_centers_
        pred_cluster = kmeans.predict(docs_vectorized)
        curr_sum_sqrd_errs = 0
        
        for i in range(len(docs_vectorized)):
            curr_center = centroids[pred_clusters[i]]
            curr_sum_sqrd_errs += (docs_vectorized[i, 0]- curr_center[0]) ** 2 + (docs_vectorized[i, 1] - curr_center[1]) ** 2
            
        sum_sqrd_errors.append(curr_sum_sqrd_erros)
    return sum_sqrd_errs